In [1]:
# GPT-2 Evaluation Notebook
!pip install transformers torch matplotlib seaborn tqdm scikit-learn pandas nltk rouge sentence-transformers

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import json
import pandas as pd
from tqdm import tqdm
import time
import random
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
random.seed(42)
torch.manual_seed(42)

# Load dataset
with open('stats_problems_dataset.json', 'r') as f:
    dataset = json.load(f)

# Stratified sampling for 300-400 problems
df = pd.DataFrame(dataset)
sampled_dataset = df.groupby('difficulty', group_keys=False).apply(
    lambda x: x.sample(min(len(x), 400 // df['difficulty'].nunique()), random_state=42)
).to_dict('records')

# Load sentence embeddings model for semantic comparison
embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Evaluation metrics
def evaluate_answer(generated, correct):
    generated_embedding = embedder.encode([generated])
    correct_embedding = embedder.encode([correct])
    similarity = cosine_similarity(generated_embedding, correct_embedding)[0][0]
    return {"is_correct": similarity > 0.7, "similarity": similarity}

# Evaluate model
def evaluate_model(model, tokenizer, dataset, device):
    results = []
    for problem in tqdm(dataset):
        prompt = f"Please solve the following statistics problem. Provide a clear, step-by-step solution, including all necessary calculations:\n\nProblem: {problem['problem']}\n\nSolution:"
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)

        try:
            start_time = time.time()
            with torch.no_grad():
                outputs = model.generate(**inputs, max_new_tokens=512, num_return_sequences=1)
            generated_solution = tokenizer.decode(outputs[0], skip_special_tokens=True)
            end_time = time.time()

            evaluation_results = evaluate_answer(generated_solution, problem['solution']['conclusion'])
            results.append({
                "model_name": "gpt2",
                "topic": problem["topic"],
                "difficulty": problem["difficulty"],
                "generated_solution": generated_solution,
                "correct_solution": problem['solution']['conclusion'],
                "time_taken": end_time - start_time,
                **evaluation_results
            })
        except Exception as e:
            print(f"Error processing problem: {e}")

    return results

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load GPT-2 model and tokenizer
model_name = "gpt2"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.eval()

# Evaluate model
results = evaluate_model(model, tokenizer, sampled_dataset, device)

# Save results to JSON
with open('gpt2_results.json', 'w') as f:
    json.dump(results, f)

print("Evaluation complete. Results saved to gpt2_results.json")
torch.cuda.empty_cache()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 20.4 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

...

Error processing problem: 'conclusion'


 90%|████████▉ | 287/320 [21:12<02:24,  4.39s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Error processing problem: 'float' object is not subscriptable


100%|██████████| 320/320 [23:38<00:00,  4.43s/it]


TypeError: Object of type bool_ is not JSON serializable

In [2]:
import numpy as np
import json

def normalize_types(results):
    """Convert numpy types to Python native types."""
    for result in results:
        for key, value in result.items():
            if isinstance(value, np.generic):  # For numpy types
                result[key] = value.item()  # Convert to Python native type
    return results

# Normalize and save results without re-running the evaluation
normalized_results = normalize_types(results)

# Save results to JSON
with open('gpt2_results.json', 'w') as f:
    json.dump(normalized_results, f)

print("Results saved to gpt2_results.json successfully.")


Results saved to gpt2_results.json successfully.


In [2]:
!pip install rouge-score sentence-transformers nltk

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 9.3 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=5fa6adcc6d62cae56fd4e5ec39eec3523f785ff72b9c2c11aa16daf4c60cc19f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [3]:
import json
import nltk
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from sentence_transformers import SentenceTransformer, util

# Load the JSON file
with open('gpt2_results (2).json', 'r') as file:
    data = json.load(file)

# Initialize models and tools
nltk.download('punkt')
rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Function to calculate BLEU, ROUGE, and Embedding Similarity
def calculate_metrics(generated_solution, correct_solution):
    # BLEU score
    reference = [nltk.word_tokenize(correct_solution)]
    hypothesis = nltk.word_tokenize(generated_solution)
    bleu_score = sentence_bleu(reference, hypothesis)

    # ROUGE score
    rouge_scores = rouge_scorer.score(correct_solution, generated_solution)
    rouge_1 = rouge_scores['rouge1'].fmeasure
    rouge_2 = rouge_scores['rouge2'].fmeasure
    rouge_l = rouge_scores['rougeL'].fmeasure

    # Embedding similarity
    embedding_gen = embedding_model.encode(generated_solution, convert_to_tensor=True)
    embedding_corr = embedding_model.encode(correct_solution, convert_to_tensor=True)
    embedding_similarity = util.pytorch_cos_sim(embedding_gen, embedding_corr).item()

    return {
        'bleu_score': bleu_score,
        'rouge_1': rouge_1,
        'rouge_2': rouge_2,
        'rouge_l': rouge_l,
        'embedding_similarity': embedding_similarity
    }

# Process the dataset and calculate the metrics
for item in data:
    generated_solution = item['generated_solution']
    correct_solution = item['correct_solution']

    metrics = calculate_metrics(generated_solution, correct_solution)

    # Add the calculated metrics to each entry
    item['bleu_score'] = metrics['bleu_score']
    item['rouge_1'] = metrics['rouge_1']
    item['rouge_2'] = metrics['rouge_2']
    item['rouge_l'] = metrics['rouge_l']
    item['embedding_similarity'] = metrics['embedding_similarity']

# Save the updated JSON file
with open('gpt2_results_with_metrics.json', 'w') as outfile:
    json.dump(data, outfile, indent=4)

print("Metrics calculation completed and saved in gpt2_results_with_metrics.json")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Metrics calculation completed and saved in gpt2_results_with_metrics.json


In [5]:
import json
import pandas as pd

# Load the JSON file
with open('gpt2_results_with_metrics.json', 'r') as file:
    data = json.load(file)

# Convert the JSON data into a DataFrame
df = pd.DataFrame(data)

# Calculate average metrics
avg_similarity = df['similarity'].mean()
avg_bleu = df['bleu_score'].mean()
avg_rouge1 = df['rouge_1'].mean()
avg_rouge2 = df['rouge_2'].mean()
avg_rouge_l = df['rouge_l'].mean()
avg_embedding_similarity = df['embedding_similarity'].mean()

# Print average metrics
print(f"Average Similarity: {avg_similarity}")
print(f"Average BLEU Score: {avg_bleu}")
print(f"Average ROUGE-1 Score: {avg_rouge1}")
print(f"Average ROUGE-2 Score: {avg_rouge2}")
print(f"Average ROUGE-L Score: {avg_rouge_l}")
print(f"Average Embedding Similarity: {avg_embedding_similarity}")

# Filter rows where the generated solution is correct
correct_solutions = df[df['is_correct'] == True]

# Display the filtered DataFrame
print(correct_solutions)

Average Similarity: 0.5971079451334589
Average BLEU Score: 0.008784097987714897
Average ROUGE-1 Score: 0.056645352999221556
Average ROUGE-2 Score: 0.0321646641989287
Average ROUGE-L Score: 0.05075769724884003
Average Embedding Similarity: 0.5971079442403294
    model_name                   topic    difficulty  \
2         gpt2                   ANOVA      advanced   
10        gpt2             Probability      advanced   
14        gpt2    Time Series Analysis      advanced   
16        gpt2    Time Series Analysis      advanced   
23        gpt2     Confidence Interval      advanced   
..         ...                     ...           ...   
297       gpt2             Probability  intermediate   
302       gpt2             Probability  intermediate   
309       gpt2  Descriptive Statistics  intermediate   
312       gpt2             Probability  intermediate   
316       gpt2      Hypothesis Testing  intermediate   

                                    generated_solution  \
2    Please